In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

( train_ds , test_ds ) , ds_info = tfds . load ('mnist', split =[ 'train', 'test'], as_supervised = True , with_info = True )
data = (train_ds, test_ds)

c:\Users\Nikod\miniconda3\envs\iannwtf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
# 70 000 images, 10 000 of which are for testing
# image shape is (28,28,1)
# image type is uint8
# value range is (0,255)
# shapeless lables with 10 different classes
def prepare_data(data):
    # flatten image into vector of values
    data = data.map(lambda img, target: (tf.reshape(img, (-1,)), target))
    # cast into continous data type
    data = data.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # normalize values into range (-1,1)
    data = data.map(lambda img, target: ((img/228.)-1., target))
    # one-hot encode labels
    data = data.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
    
    return data
    
training_data = train_ds.apply(prepare_data)
testing_data = prepare_data(test_ds)
# print(data)
print (training_data)
# print (testing_data)

    # # shuffle
    # data = data.shuffle(1000)
    # # batch 
    # data = data.batch(32)
    # # prefetch
    # data = data.prefetch(20)
    # # save data
    # data = data.cache()

# Create a `Sequential` model and add a Dense layer as the first layer.
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape = (784), name='input'))
model.add(tf.keras.layers.Dense(units=32, activation='relu', name='d1'))
model.add(tf.keras.layers.Dense(units=32, activation='relu', name='d2'))
model.add(tf.keras.layers.Dense(units=10, name='final'))




def train_network(n_epochs, model, train_ds, test_ds, loss, optimizer, learning_rate, metrics):
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.fit(x=train_ds, epochs=n_epochs, validation_data=test_ds)


<MapDataset element_spec=(TensorSpec(shape=(784,), dtype=tf.float32, name=None), TensorSpec(shape=(10,), dtype=tf.float32, name=None))>


In [50]:
train_network(
    n_epochs=10, model=model, learning_rate=0.1,train_ds=training_data,
    test_ds=testing_data, loss=tf.keras.losses.CategoricalCrossentropy(), 
    optimizer=tf.keras.optimizers.SGD(), metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

Epoch 1/10


ValueError: in user code:

    File "c:\Users\Nikod\miniconda3\envs\iannwtf\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Nikod\miniconda3\envs\iannwtf\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Nikod\miniconda3\envs\iannwtf\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Nikod\miniconda3\envs\iannwtf\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Nikod\miniconda3\envs\iannwtf\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Nikod\miniconda3\envs\iannwtf\lib\site-packages\keras\engine\input_spec.py", line 228

In [ ]:
def visualization ( train_losses, train_accuracies, test_losses, test_accuracies):
    """ Visualizes accuracy and loss for training and test data using
    the mean of each epoch .
    Loss is displayed in a regular line , accuracy in a dotted
    line .
    Training data is displayed in blue , test data in red .
    Parameters
    ----------
    train_losses : numpy . ndarray
    training losses
    train_accuracies : numpy . ndarray
    training accuracies
    test_losses : numpy . ndarray
    test losses
    test_accuracies : numpy . ndarray
    test accuracies"""
    plt.figure ()
    line1, = plt.plot( train_losses , "b-")
    line2, = plt.plot( test_losses , "r-")
    line3, = plt.plot( train_accuracies , "b:")
    line4, = plt.plot( test_accuracies , "r:")
    plt.xlabel(" Training steps ")
    plt.ylabel(" Loss / Accuracy ")
    plt.legend((line1, line2, line3, line4), 
    (" training loss ", " test loss ", " train accuracy ", " test accuracy "))
    plt.show()
